In [46]:
# 🚀 **Test API Calls**
if __name__ == "__main__":
    import pandas as pd
    api = SimFinAPI("b7f5ad1b-6cd9-4f19-983b-cfddaad8df9c")

    # **Test get_share_prices**
    print("\nFetching Apple (AAPL) Share Prices from past year...")
    share_prices_df = api.get_share_prices("AAPL", "2023-12-30", "2025-03-06")
    print(share_prices_df.head(10) if not share_prices_df.empty else "No share price data retrieved.")

    # **Test get_income_statement**
    print("\nFetching Apple (AAPL) Income Statement from past year...")
    income_df = api.get_income_statement("AAPL", "2023-12-30", "2025-03-06")
    print(income_df.head(10) if not income_df.empty else "No income statement data retrieved.")

    # **Test get_balance_sheet**
    print("\nFetching Apple (AAPL) Balance Sheet from past year...")
    balance_sheet_df = api.get_balance_sheet("AAPL", "2023-12-30", "2025-03-06")
    print(balance_sheet_df.head(10) if not balance_sheet_df.empty else "No balance sheet data retrieved.")


Fetching Apple (AAPL) Share Prices from past year...
        date ticker   close
0 2024-01-02   AAPL  185.64
1 2024-01-03   AAPL  184.25
2 2024-01-04   AAPL  181.91
3 2024-01-05   AAPL  181.18
4 2024-01-08   AAPL  185.56
5 2024-01-09   AAPL  185.14
6 2024-01-10   AAPL  186.19
7 2024-01-11   AAPL  185.59
8 2024-01-12   AAPL  185.92
9 2024-01-16   AAPL  183.63

Fetching Apple (AAPL) Income Statement from past year...
  ticker       date fiscal_period  fiscal_year       revenue    netIncome
0   AAPL 2024-06-30            9M         2024  296105000000  79000000000
1   AAPL 2024-09-30            FY         2024  391035000000  93736000000
2   AAPL 2024-03-31            H1         2024  210328000000  57552000000
3   AAPL 2024-09-30            H2         2024  180707000000  36184000000
4   AAPL 2023-12-31            Q1         2024  119575000000  33916000000
5   AAPL 2024-12-31            Q1         2025  124300000000  36330000000
6   AAPL 2024-03-31            Q2         2024   90753000000  

In [44]:
print(f"Share Prices DataFrame loaded from API or file: {share_prices_df.shape}")
print(f"Income Statement DataFrame loaded from API or file: {income_df.shape}")
print(f"Balance Sheet DataFrame loaded from API or file: {balance_sheet_df.shape}")

Share Prices DataFrame loaded from API or file: (295, 3)
Income Statement DataFrame loaded from API or file: (9, 6)
Balance Sheet DataFrame loaded from API or file: (5, 4)


In [31]:

# Convert date columns to datetime format for proper merging
balance_sheet_df["date"] = pd.to_datetime(balance_sheet_df["date"])
income_df["date"] = pd.to_datetime(income_df["date"])
share_prices_df["date"] = pd.to_datetime(share_prices_df["date"])

# Merge datasets using a left join on ticker and date
merged_df = share_prices_df.merge(income_df, on=["ticker", "date"], how="left")
merged_df = merged_df.merge(balance_sheet_df, on=["ticker", "date"], how="left")

# Sort by ticker and date (most recent to oldest)
merged_df = merged_df.sort_values(by=["date"], ascending=[True])

# Forward-fill missing values for financial data
merged_df.fillna(method="ffill", inplace=True)

# Compute P/E ratio (Price-to-Earnings Ratio)
merged_df["earnings_per_share"] = merged_df["net_income"] / merged_df["share_capital"]
merged_df["p_e_ratio"] = merged_df["close"] / merged_df["earnings_per_share"]

# Compute 50-day Simple Moving Average (SMA)
merged_df["sma_50"] = merged_df.groupby("ticker")["close"].transform(lambda x: x.rolling(window=50, min_periods=1).mean())

# Add next day's close price as a target variable
merged_df["next_close"] = merged_df.groupby("ticker")["close"].shift(-1)

# Drop rows where critical features contain NaN values
merged_df = merged_df.dropna(subset=["close", "p_e_ratio", "sma_50"])

# Display the first few rows of the final dataset
display(merged_df)

KeyError: 'net_income'